# [TUTORIAL] Constrained Task Planning (Sweep)

This tutorial is written to instruct basic usage of the task & motion planning pipeline  
One Indy7 robot and several environment geometries will be added and floor-wiping task will be conducted.  
Here, the wiping task is defined as 1) contact with floor, 2) starting waypoint, 3) goal waypoint.  
Thus, any motion that satisfies the constraint will be generated; it may not look like real wiping.  

## set running directory to project source

In [1]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

## init combined robot config

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, None,
                INDY_IP)]
              , connection_list=[False])

connection command:
indy0: False


## create scene builder

In [3]:
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None)
# s_builder.reset_reference_coord(ref_name="floor")

## get ghnd with detected robot config

In [4]:
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
xyz_rpy_robots = {"indy0": ((0,0,0), (0,0,-np.pi/2))}
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)

Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]


## add environment

In [5]:
from pkg.geometry.geometry import *

floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (3,3,0.01), (0,0,0.001), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=False, fixed=True, collision=False)
floor = gscene.create_safe(GEOTYPE.BOX, "floor_viscol", "base_link", (3,3,0.01), (0,0,0), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
wall = gscene.create_safe(GEOTYPE.BOX, "wall", "base_link", (3,3,0.01), (0,0.2,0), 
                           rpy=(np.pi/2,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
wp1 = gscene.create_safe(GEOTYPE.BOX, "wp1", "base_link", (0.1,0.1,0.01), (0.2,-0.5,0.001),rpy=(0,0,np.pi/2), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False)
wp2 = gscene.create_safe(GEOTYPE.BOX, "wp2", "base_link", (0.1,0.1,0.01), (-0.2,-0.5,0.001), rpy=(0,0,np.pi/2), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False)
track = gscene.create_safe(GEOTYPE.BOX, "track", "base_link", (0.1,0.5,0.01), (0,-0.5,0), rpy=(0,0,np.pi/2), 
                           color=(1,0.7,0.7,0.5), display=True, fixed=True, collision=False)
goal = gscene.create_safe(GEOTYPE.BOX, "goal", "base_link", (0.1,0.1,0.01), (-0.4,-0.3,0.001), 
                          rpy=(0,0,0), color=(0.8,0.8,0.2,1), display=True, fixed=True, collision=False)

Please create a subscriber to the marker


In [6]:
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True, exclude_link=["panda1_link7"])

In [7]:
from pkg.utils.code_scraps import add_indy_gripper_asm2
add_indy_gripper_asm2(gscene, "indy0", "indy0_tcp")

## add brush

In [8]:
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_body", link_name="base_link", dims=(0.2,0.07,0.02), 
                   center=(0,-0.5,0.045), rpy=(0,0,np.pi/2), color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=False)
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_handle", link_name="base_link", dims=(0.2,0.03,0.05), center=(0,0,0.035), rpy=(0,0,0), 
                   color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=False, parent="brush_body")
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_face", link_name="base_link", dims=(0.19,0.06,0.03), center=(0,0,-0.025), rpy=(0,0,0), 
                   color=(0.8,0.8,0.8,1), display=True, collision=False, fixed=False, parent="brush_body")

## add box

In [9]:
gscene.create_safe(gtype=GEOTYPE.BOX, name="obstacle", link_name="base_link", dims=(0.05,0.05,0.05), 
                   center=(0.2,-0.5,0.031), rpy=(0,0,0), color=(0.7,0.3,0.3,1), display=True, collision=True, fixed=False)

## init planning scene

In [10]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

## Register binders

In [11]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepTool

In [12]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="indy0_tcp", 
                 dims=(0.01,)*3, center=(0,0,0.14), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

In [13]:
pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane)
pscene.create_binder(bname="goal_bd", gname="goal", _type=PlacePlane, point=(0,0,0.005), rpy=(0,0,0))
pscene.create_binder(bname="brush_face", gname="brush_face", _type=SweepTool, point=(0,0,-0.015), rpy=(0,0,0))

## add objects

In [14]:
from pkg.planning.constraint.constraint_subject import CustomObject, Grasp2Point, PlacePoint, SweepPoint, SweepTask, BoxObject

In [15]:
brush_handle = gscene.NAME_DICT["brush_handle"]
brush_face = gscene.NAME_DICT["brush_face"]
brush = pscene.create_subject(oname="brush", gname="brush_body", _type=CustomObject, 
                             action_points_dict = {"handle": Grasp2Point("handle", brush_handle, [0,0,0], [np.pi/2,0,0]),
                                                   "face": PlacePoint("face", brush_face, [0,0,-0.015], [0,0,0])})

In [16]:
obstacle = pscene.create_subject(oname="obstacle", gname="obstacle", _type=BoxObject, hexahedral=True)

In [17]:
sweep = pscene.create_subject(oname="sweep", gname="floor", _type=SweepTask, 
                             action_points_dict = {"wp1": SweepPoint("wp1", wp1, [0,0,0.005], [0,0,0]),
                                                   "wp2": SweepPoint("wp2", wp2, [0,0,0.005], [0,0,0])})

### planning pipeline

In [18]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)
mplan.update_gscene()
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()
from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

## motion filters

In [20]:
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
from pkg.planning.filtering.latticized_filter import LatticedChecker
TOOL_LINK = "indy0_tcp"
TOOL_LINK_BUNDLE = ["indy0_tcp", "indy0_link6"]

gcheck = GraspChecker(pscene)
rcheck = ReachChecker(pscene)
checkers_all = [gcheck, rcheck]
# lcheck = LatticedChecker(pscene, gcheck)
# checkers_all.append(lcheck)

In [21]:
mplan.motion_filters = checkers_all

## ui

In [22]:
from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


```
open web ui on <your ip>:8050
click geometry items / Handles / Binders to highlight geometry on RVIZ
other functions may be buggy.. please report
```

## Test plan

In [23]:
from pkg.planning.constraint.constraint_common \
            import sample_redundancy, combine_redundancy
gtimer = GlobalTimer.instance()
initial_state = pscene.update_state(crob.home_pose)

In [24]:
gtimer.reset()
for _ in range(10):
    # goal_nodes = [("goal",)]
    goal_nodes = [("goal", "floor", 2)]
    gtimer.tic("plan")
    ppline.search(initial_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, timeout_loop=300, multiprocess=True)
    gtimer.toc("plan")
    schedules = ppline.tplan.find_schedules()
    schedules_sorted = ppline.tplan.sort_schedule(schedules)
    snode_schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])

Use 20/20 agents
try: 0 - ('floor', 'floor', 0)->('floor', 'floor', 1)
result: 0 - ('floor', 'floor', 0)->('floor', 'floor', 1) = fail
try: 0 - ('floor', 'floor', 0)->('floor', 'grip0', 0)
try: 0 - ('floor', 'floor', 0)->('grip0', 'floor', 0)
try transition motion
transition motion tried: True
result: 0 - ('floor', 'floor', 0)->('floor', 'grip0', 0) = success
result: 0 - ('floor', 'floor', 0)->('grip0', 'floor', 0) = fail
try: 1 - ('floor', 'grip0', 0)->('floor', 'grip0', 1)
branching: 0->1 (0.09/300.0 s, steps/err: 17(36.1058712006 ms)/0.00149832553107)
result: 1 - ('floor', 'grip0', 0)->('floor', 'grip0', 1) = fail
try: 0 - ('floor', 'floor', 0)->('floor', 'floor', 1)
try: 0 - ('floor', 'floor', 0)->('grip0', 'floor', 0)
result: 0 - ('floor', 'floor', 0)->('floor', 'floor', 1) = fail
result: 0 - ('floor', 'floor', 0)->('grip0', 'floor', 0) = fail
try: 0 - ('floor', 'floor', 0)->('floor', 'floor', 1)
try: 0 - ('floor', 'floor', 0)->('floor', 'floor', 1)
result: 0 - ('floor', 'floor', 

transition motion tried: True
try: 0 - ('floor', 'floor', 0)->('floor', 'grip0', 0)
result: 4 - ('grip0', 'floor', 0)->('goal', 'floor', 0) = success
try transition motion
branching: 4->6 (0.44/300.0 s, steps/err: 26(40.3919219971 ms)/0.00191032585449)
try: 6 - ('goal', 'floor', 0)->('goal', 'floor', 1)
transition motion tried: True
result: 1 - ('floor', 'grip0', 0)->('floor', 'floor', 0) = fail
result: 1 - ('floor', 'grip0', 0)->('floor', 'goal', 0) = success
try: 7 - ('floor', 'goal', 0)->('floor', 'goal', 1)
result: 0 - ('floor', 'floor', 0)->('floor', 'grip0', 0) = fail
try: 6 - ('goal', 'floor', 0)->('goal', 'grip0', 0)
branching: 1->7 (0.48/300.0 s, steps/err: 35(82.346200943 ms)/0.00178322077077)
transition motion tried: True
try: 2 - ('floor', 'grip0', 0)->('floor', 'grip0', 1)
result: 2 - ('floor', 'grip0', 0)->('floor', 'grip0', 1) = fail
result: 7 - ('floor', 'goal', 0)->('floor', 'goal', 1) = fail
result: 6 - ('goal', 'floor', 0)->('goal', 'grip0', 0) = fail
try: 1 - ('floo

try: 13 - ('goal', 'floor', 0)->('goal', 'grip0', 0)
result: 0 - ('floor', 'floor', 0)->('floor', 'floor', 1) = fail
try transition motion
result: 2 - ('floor', 'grip0', 0)->('floor', 'floor', 0) = fail
try: 13 - ('goal', 'floor', 0)->('goal', 'floor', 1)
try: 0 - ('floor', 'floor', 0)->('floor', 'floor', 1)
try: 10 - ('floor', 'grip0', 0)->('floor', 'floor', 0)
result: 0 - ('floor', 'floor', 0)->('floor', 'grip0', 0) = success
try transition motion
try: 9 - ('floor', 'grip0', 0)->('floor', 'floor', 0)
try: 12 - ('goal', 'floor', 0)->('grip0', 'floor', 0)
try transition motion
result: 13 - ('goal', 'floor', 0)->('goal', 'floor', 1) = fail
result: 0 - ('floor', 'floor', 0)->('grip0', 'floor', 0) = fail
result: 7 - ('floor', 'goal', 0)->('floor', 'goal', 1) = fail
try: 12 - ('goal', 'floor', 0)->('goal', 'floor', 1)
try: 16 - ('goal', 'floor', 0)->('goal', 'floor', 1)
result: 3 - ('grip0', 'floor', 0)->('goal', 'floor', 0) = success
result: 10 - ('floor', 'grip0', 0)->('floor', 'floor', 

result: 2 - ('floor', 'grip0', 0)->('floor', 'grip0', 1) = fail
result: 6 - ('goal', 'floor', 0)->('goal', 'floor', 1) = fail
try: 16 - ('goal', 'floor', 0)->('goal', 'grip0', 0)
try: 0 - ('floor', 'floor', 0)->('grip0', 'floor', 0)
try transition motion
result: 0 - ('floor', 'floor', 0)->('grip0', 'floor', 0) = fail
transition motion tried: True
try: 3 - ('grip0', 'floor', 0)->('grip0', 'floor', 1)
branching: 16->19 (1.06/300.0 s, steps/err: 11(59.4489574432 ms)/0.00112976504391)
try: 20 - ('goal', 'floor', 0)->('goal', 'floor', 1)
result: 4 - ('grip0', 'floor', 0)->('goal', 'floor', 0) = success
result: 0 - ('floor', 'floor', 0)->('grip0', 'floor', 0) = fail
result: 19 - ('goal', 'grip0', 0)->('goal', 'floor', 0) = fail
result: 16 - ('goal', 'floor', 0)->('goal', 'grip0', 0) = fail
result: 3 - ('grip0', 'floor', 0)->('goal', 'floor', 0) = success
branching: 3->20 (1.07/300.0 s, steps/err: 9(101.552009583 ms)/0.000635727880896)
result: 14 - ('floor', 'goal', 0)->('grip0', 'goal', 0) =

result: 12 - ('goal', 'floor', 0)->('goal', 'grip0', 0) = fail
try transition motion
try: 19 - ('goal', 'grip0', 0)->('goal', 'grip0', 1)
try: 8 - ('floor', 'grip0', 0)->('floor', 'floor', 0)
result: 19 - ('goal', 'grip0', 0)->('goal', 'goal', 0) = fail
try: 14 - ('floor', 'goal', 0)->('floor', 'grip0', 0)
try: 26 - ('floor', 'goal', 0)->('floor', 'goal', 1)
result: 19 - ('goal', 'grip0', 0)->('goal', 'grip0', 1) = fail
try: 0 - ('floor', 'floor', 0)->('floor', 'floor', 1)
result: 1 - ('floor', 'grip0', 0)->('floor', 'grip0', 1) = fail
try: 17 - ('goal', 'floor', 0)->('goal', 'floor', 1)
result: 10 - ('floor', 'grip0', 0)->('floor', 'goal', 0) = fail
transition motion tried: True
result: 9 - ('floor', 'grip0', 0)->('floor', 'goal', 0) = success
try: 18 - ('goal', 'grip0', 0)->('goal', 'goal', 0)
result: 18 - ('goal', 'grip0', 0)->('goal', 'goal', 0) = fail
try: 11 - ('grip0', 'floor', 0)->('grip0', 'floor', 1)
try: 9 - ('floor', 'grip0', 0)->('floor', 'grip0', 1)
try: 7 - ('floor', 'go

...

try: 54 - ('grip0', 'goal', 2)->('goal', 'goal', 2)
try: 68 - ('grip0', 'floor', 1)->('goal', 'floor', 1)
try: 1 - ('floor', 'grip0', 0)->('floor', 'goal', 0)
try transition motion
result: 54 - ('grip0', 'goal', 2)->('goal', 'goal', 2) = fail
try: 44 - ('floor', 'goal', 2)->('floor', 'grip0', 2)
result: 35 - ('grip0', 'goal', 1)->('goal', 'goal', 1) = fail
result: 1 - ('floor', 'grip0', 0)->('floor', 'goal', 0) = fail
try: 29 - ('grip0', 'floor', 0)->('grip0', 'floor', 1)
try: 68 - ('grip0', 'floor', 1)->('grip0', 'floor', 2)
try: 48 - ('floor', 'goal', 1)->('floor', 'goal', 2)
result: 44 - ('floor', 'goal', 2)->('floor', 'grip0', 2) = fail
result: 29 - ('grip0', 'floor', 0)->('grip0', 'floor', 1) = fail
try: 48 - ('floor', 'goal', 1)->('floor', 'grip0', 1)
result: 48 - ('floor', 'goal', 1)->('floor', 'goal', 2) = fail
try constrained motion
try: 0 - ('floor', 'floor', 0)->('grip0', 'floor', 0)
result: 48 - ('floor', 'goal', 1)->('floor', 'grip0', 1) = fail
result: 0 - ('floor', 'floor

transition motion tried: False
transition motion tried: False
result: 70 - ('floor', 'grip0', 2)->('floor', 'floor', 2) = fail
try: 27 - ('goal', 'grip0', 0)->('goal', 'grip0', 1)
try: 57 - ('grip0', 'goal', 1)->('grip0', 'goal', 2)
result: 25 - ('goal', 'grip0', 0)->('goal', 'grip0', 1) = fail
try: 15 - ('goal', 'floor', 0)->('goal', 'grip0', 0)
result: 4 - ('floor', 'goal', 0)->('grip0', 'goal', 0) = fail
try: 1 - ('floor', 'grip0', 0)->('floor', 'grip0', 1)
try: 77 - ('grip0', 'goal', 2)->('floor', 'goal', 2)
result: 74 - ('floor', 'goal', 2)->('floor', 'grip0', 2) = fail
result: 27 - ('goal', 'grip0', 0)->('goal', 'grip0', 1) = fail
try constrained motion
try: 10 - ('floor', 'floor', 0)->('floor', 'floor', 1)
try: 47 - ('grip0', 'goal', 0)->('floor', 'goal', 0)
transition motion tried: True
result: 15 - ('goal', 'floor', 0)->('goal', 'grip0', 0) = fail
result: 1 - ('floor', 'grip0', 0)->('floor', 'grip0', 1) = fail
try transition motion
result: 10 - ('floor', 'floor', 0)->('floor',

try: 65 - ('grip0', 'floor', 0)->('floor', 'floor', 0)
try: 50 - ('floor', 'grip0', 2)->('floor', 'goal', 2)
try: 91 - ('goal', 'floor', 0)->('goal', 'floor', 1)
result: 50 - ('floor', 'grip0', 2)->('floor', 'goal', 2) = fail
try: 92 - ('grip0', 'floor', 1)->('grip0', 'floor', 2)
try: 82 - ('goal', 'floor', 1)->('goal', 'floor', 2)
result: 59 - ('floor', 'floor', 0)->('floor', 'floor', 1) = fail
result: 55 - ('grip0', 'floor', 0)->('grip0', 'floor', 1) = success
result: 65 - ('grip0', 'floor', 0)->('floor', 'floor', 0) = fail
try: 76 - ('grip0', 'floor', 0)->('goal', 'floor', 0)
result: 91 - ('goal', 'floor', 0)->('goal', 'floor', 1) = fail
try: 54 - ('grip0', 'goal', 2)->('floor', 'goal', 2)
result: 82 - ('goal', 'floor', 1)->('goal', 'floor', 2) = fail
try transition motion
try: 55 - ('grip0', 'floor', 0)->('floor', 'floor', 0)
try: 81 - ('grip0', 'goal', 0)->('grip0', 'goal', 1)
try transition motion
result: 54 - ('grip0', 'goal', 2)->('floor', 'goal', 2) = fail
try constrained moti

try: 99 - ('goal', 'floor', 2)->('goal', 'grip0', 2)
try: 68 - ('grip0', 'floor', 1)->('grip0', 'floor', 2)
result: 18 - ('grip0', 'goal', 1)->('goal', 'goal', 1) = fail
branching: 97->102 (6.25/300.0 s, steps/err: 11(71.7658996582 ms)/0.00101161765261)
try: 103 - ('grip0', 'floor', 1)->('floor', 'floor', 1)
try transition motion
result: 47 - ('grip0', 'goal', 0)->('grip0', 'goal', 1) = success
try constrained motion
result: 103 - ('grip0', 'floor', 1)->('floor', 'floor', 1) = fail
try: 48 - ('floor', 'goal', 1)->('grip0', 'goal', 1)
branching: 47->101 (6.28/300.0 s, steps/err: 27(1335.44707298 ms)/0.00179968150007)
++ adding return motion to acquired answer ++
try joint motion
try: 85 - ('floor', 'floor', 1)->('floor', 'grip0', 1)
joint motion tried: True
try: 59 - ('floor', 'floor', 0)->('grip0', 'floor', 0)
transition motion tried: True
result: 82 - ('goal', 'floor', 1)->('grip0', 'floor', 1) = success
try: 82 - ('goal', 'floor', 1)->('goal', 'grip0', 1)
try transition motion
branch

======================= terminated 13: first answer acquired from other agent ===============================
======================= terminated 5: first answer acquired from other agent ===============================
======================= terminated 3: first answer acquired ===============================
transition motion tried: False
result: 27 - ('goal', 'grip0', 0)->('goal', 'floor', 0) = fail
======================= terminated 7: first answer acquired from other agent ===============================
constrained motion tried: False
result: 92 - ('grip0', 'floor', 1)->('grip0', 'floor', 2) = fail
======================= terminated 2: first answer acquired from other agent ===============================
transition motion tried: False
result: 55 - ('grip0', 'floor', 0)->('floor', 'floor', 0) = fail
======================= terminated 15: first answer acquired from other agent ===============================
constrained motion tried: False
result: 68 - ('grip0', 'floor', 1)->('grip

In [25]:
print(gtimer)

plan: 	79987.0 ms/10 = 7998.706 ms (3799.539/11618.183)



## play searched plan

In [26]:
ppline.play_schedule(snode_schedule[:2], period=0.01)
pscene.set_object_state(initial_state)
gscene.show_pose(crob.home_pose)
time.sleep(0.5)
gscene.show_pose(crob.home_pose)

In [28]:
ppline.play_schedule(snode_schedule, period=0.03)